In [1]:
# import basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

# import stuff from the mixture_composition_regression package
from mixture_composition_regression.examples.load_dipa_water_nacl_training_set import load_training_set
from mixture_composition_regression.cross_validation import cv_on_model_and_wavelength
from mixture_composition_regression.import_spectrum import clean_data
from mixture_composition_regression.sample import Sample
from mixture_composition_regression.mixture import Mixture
from mixture_composition_regression.preprocessor_pipeline import get_Xy
from mixture_composition_regression.gridsearch_dataset import grid_search_dataset

# import needed packages from scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.tree import DecisionTreeRegressor

In [2]:
# read and clean training data
file = '/Users/ianbillinge/dev/mixture_composition_regression/mixture_composition_regression/examples/bromide/Bromide Salts NIR DIPA Measurements - salt+water stds.csv'
df = clean_data(file)

x = df['Wavelength (nm)']
# abs_cols = [col for col in df.columns if 'Abs' in col]
# print(abs_cols)
df = df.filter(regex='Abs') # retain just the ab
samples = df.columns

ds = xr.Dataset({'a':(['x', 'sample'], df)}, coords = {'x':x,
#                                                        'sample': names
                                                      })

# read and clean target data (weight fraction).
# w_file = '/Users/ianbillinge/Documents/yiplab/projects/ir/cellulose/composition.csv'
# composition = pd.read_csv(w_file)

# samples = np.array(composition.columns)[1:]

cp = {'name': ['water', 'dipa', 'NaBr'],
      'mw': [18.015, 101.19, 
             1 ### update
            ],
      'nu': [1, 1, 2]}

w = np.random.rand(3, df.columns.shape[0])

ds = ds.assign_coords({'chems': (cp['name'])})
ds = ds.assign_coords(
    w_water = ('samples', w[0,:]),         
    w_dipa = ('samples', w[1,:]),          
    w_nabr = ('samples', w[2, :])
                     )
ds['da'] = ds['a'].diff('x').dropna('x', how='any') # something funny is happening with dropna
ds['d2a'] = ds['a'].diff('x', 2).dropna('x', how='any')

# # # create a list of mixture_composition_regression.Sample objects (one for each spectrum you collected)
# ds = []
# for s in samples:
#     ds.append(Sample(s, df, x_col_name='Wavelength (nm)', a_col_name=s, 
#                      chem_properties=cp, w=list(composition[s]/100.),
#                     xbounds = [500, 3900]))

# # create a mixture_composition_regression.Mixture object
# # NOTE: I will probably remove the Mixture object from the package in the future, 
# # and I do the regression without using the Mixture, but there are some plotting things that are quite nice.
# mix = Mixture(ds)

In [3]:
ds

<xarray.Dataset>
Dimensions:  (x: 2801, chems: 3, sample: 23, samples: 23)
Coordinates:
  * x        (x) float64 3.2e+03 3.199e+03 3.198e+03 ... 402.0 401.0 400.0
  * chems    (chems) <U5 'water' 'dipa' 'NaBr'
    w_water  (samples) float64 0.2338 0.2675 0.8572 ... 0.2447 0.9903 0.8044
    w_dipa   (samples) float64 0.8811 0.2403 0.1458 ... 0.8407 0.1456 0.6401
    w_nabr   (samples) float64 0.9455 0.6013 0.9764 ... 0.2216 0.1316 0.6438
Dimensions without coordinates: sample, samples
Data variables:
    a        (x, sample) float64 10.0 10.0 10.0 10.0 ... 0.06412 0.05553 0.05374
    da       (x, sample) float64 nan nan nan nan ... -8.5e-05 -9.5e-05 -8.8e-05
    d2a      (x, sample) float64 nan nan nan nan ... 5.5e-05 -0.000136 1.4e-05

In [4]:
sc = 'neg_mean_absolute_error' # other scoring methods are available... 
                               # see anything under 'Regression' at scikit-learn.org/stable/modules/model_evaluation.html
cv_number = 5

In [5]:
ridge_param_grid = {'alpha': np.logspace(-7, 7, 14)}

ridge = GridSearchCV(
    Ridge(), 
    param_grid = ridge_param_grid, 
    scoring=sc, 
    cv=cv_number
)


kr_param_grid = {'kernel': ["rbf", 'linear'],
                "alpha": np.logspace(-7, 7, 11),
                "gamma": np.logspace(-7, 7, 11)
                }

kr = GridSearchCV(
    KernelRidge(),
    param_grid=kr_param_grid,
    scoring=sc,
    cv = cv_number
)

svr_param_grid = {'kernel': ['linear', 
#                              'rbf'
                            ],
     'gamma': ['scale', 'auto'],
     'epsilon': np.logspace(-7, 7, 10)
     }

svr = GridSearchCV(
    SVR(),
    param_grid=svr_param_grid,
    scoring=sc,
    cv = cv_number
)

knnr_param_grid = {'n_neighbors': 5 + np.arange(5)}
knnr = GridSearchCV(
    KNeighborsRegressor(), param_grid=knnr_param_grid, scoring=sc
)

mlp = GridSearchCV(
    MLPRegressor(solver='lbfgs', max_iter=400),
    param_grid = {'hidden_layer_sizes': [10, 50, 100]},
    scoring=sc,
    cv=cv_number
)

pls = GridSearchCV(
    PLSRegression(),
    param_grid = {'n_components': [2, 4, 6, 8]},
    scoring=sc,
    cv=cv_number

)

dtr = GridSearchCV(
    DecisionTreeRegressor(),
    param_grid = {'max_depth': [2, 
#                                 3, 
#                                 5
                               ],
                  'min_samples_split': [2, 3]},
    scoring = sc,
    cv=cv_number
)

In [6]:
nwindows = [1, 
#             5,
#             10, 
#             50, 
#             200
           ]

In [7]:
random_state = 1
# random_state = None # specify a replicable random split; this line can be set to None if not desired.
tts_size = 0.25 # reserve 1/4 of the data for testing.

# Specify a metric by which to compare models. Currently set to mean absolute error.
# Other metrics available at scikit-learn.org/stable/modules/model_evaluation.html

metric = mean_absolute_error
metric_label = 'MAE'

In [8]:
cv_models = [
    ridge,
#     pls,
#     dtr,
#     kr,
#     svr,
#     knnr,
#     mlp,
]

In [9]:
yname = 'da' # 
# yname = 'd2a'
mix_train, mix_test = train_test_split(ds[yname], 
                                       test_size=0.2, 
                                       random_state=1
                                      )

In [10]:
viable_models, best_model, y_best, X_best = grid_search_dataset(
    mix_train,
    nwindows,
    cv_models,
    target_chem='dipa',
#     test_data=mix_test,
    test_data=None, # random selection of test data
    tts_test_size=tts_size,
    tts_random_state=random_state,
    tolerance=0.01,
    metric=metric,
    metric_label=metric_label,
    x_bounds=None, # here is where you could restrict your spectral range
    plot_comparison=True,
    plot_comparison_savefile='./plots/axes_train'
)

Running analysis splitting interval into 1 windows.
Running analysis on Ridge()
xbounds is not a DataArray or Dataset.
xbounds: [ 400. 3200.]
bds:
type: <class 'numpy.ndarray'>
shape: (2240,)
[ True  True  True ...  True  True  True]


ValueError: operands could not be broadcast together with shapes (2240,) (2240,23) () 

In [11]:
ds

<xarray.Dataset>
Dimensions:  (x: 2801, chems: 3, sample: 23, samples: 23)
Coordinates:
  * x        (x) float64 3.2e+03 3.199e+03 3.198e+03 ... 402.0 401.0 400.0
  * chems    (chems) <U5 'water' 'dipa' 'NaBr'
    w_water  (samples) float64 0.2338 0.2675 0.8572 ... 0.2447 0.9903 0.8044
    w_dipa   (samples) float64 0.8811 0.2403 0.1458 ... 0.8407 0.1456 0.6401
    w_nabr   (samples) float64 0.9455 0.6013 0.9764 ... 0.2216 0.1316 0.6438
Dimensions without coordinates: sample, samples
Data variables:
    a        (x, sample) float64 10.0 10.0 10.0 10.0 ... 0.06412 0.05553 0.05374
    da       (x, sample) float64 nan nan nan nan ... -8.5e-05 -9.5e-05 -8.8e-05
    d2a      (x, sample) float64 nan nan nan nan ... 5.5e-05 -0.000136 1.4e-05